In [ ]:
from utils.query_model import *
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import time
import sys
import re

In [ ]:
df = pd.read_csv('../data/guland_hanoi_listings_arcgis3.csv')

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
t, p, a, d, lat, lon, pm2, nl = "Title", "Price", "Area", "Description", "Latitude", "Longitude", "Price per m2", "\n"

def create_general_info(df):
    # Convert Description NaNs to empty string
    description = df[d].fillna("Không có mô tả").astype(str)
    
    # prince_info = "Giá nhà: " + (df[p]/1000000).astype(str) + "tỷ" + nl
    area_info = "Diện tích: " + df[a].astype(str) + "m2" + nl
    # lat_info = "Vĩ độ: " + df[lat].astype(str) + nl
    # lon_info = "Kinh độ: " + df[lon].astype(str) + nl
    # pm2_info = "Giá/m2: " + (df[pm2]).astype(str) + " triệu" + nl

    return df[t] + nl + description + nl + area_info #+  prince_info + lat_info + lon_info + pm2_info

In [ ]:
model = "Qwen/Qwen3-0.6B-Base"
device = "auto"
max_new_tokens = 5
# batch_inference = True
# batch_size = 20
temperature = 1.0
top_p = 1.0
length_df = df.shape[0]
max_retries = 3

In [ ]:
sys_prompt = """Bạn là chuyên gia phân tích bất động sản Việt Nam. 
Nhiệm vụ của bạn là xác định số phòng ngủ từ mô tả bất động sản.

Quy trình phân tích:
1. Tìm các cụm từ chính xác như: " A phòng ngủ", "APN", "A PN", "A Phòng Ngủ", "A PHÒNG NGỦ"
2. Nếu thấy nhiều con số khác nhau, ưu tiên số được ghi kèm với "phòng ngủ" hoặc "PN"
3. Nếu không tìm thấy thông tin rõ ràng, KHÔNG đưa ra giả định

VÍ DỤ:
"Nhà có 3 phòng ngủ rộng rãi" → 3
"Căn hộ 2PN" → 2
"Nhà 4 phòng (2 phòng ngủ, 1 phòng khách)" → 2
"Diện tích 80m2" → KHÔNG đủ thông tin để xác định -> 0

LƯU Ý QUAN TRỌNG:
- Hạn chế trả lời số 0, cố gắng suy đoán thông tin về phòng ngủ
- CHỈ trả lời MỘT chữ số từ 0-9
- KHÔNG giải thích hoặc thêm từ nào khác
"""

In [ ]:
df_new = pd.DataFrame()
df_new['general_info'] = create_general_info(df)
df_new.info()

In [ ]:
df_new.head()

In [ ]:
df_new['category'] = -1
df_new['retry_count'] = 0

df_new.info()

In [ ]:
interface = ModelQueryInterface()
interface.load_model(model_name=model, device_map=device)

def check_valid_room_number_response(response):
    # Clean response more thoroughly
    response = response.strip().lower()
    # First check for exact single digit
    if re.search(r'^[1-9]$', response):
        return int(response)
    # Second try to extract first digit
    match = re.search(r'[1-9]', response)
    if match:
        return int(match.group(0))
    return None

total_processed = 0
total_errors = 0
error_rate = 0.0

for i in range(max_retries):
    progress_bar = tqdm(range(length_df), desc=f"Retry {i+1}/{max_retries} | Error rate: {error_rate:.2f}%")
    for idx in progress_bar:
        if df_new.loc[idx, 'category'] == -1:
            prompt = f"{sys_prompt}\nThông tin bất động sản:\n{df_new.loc[idx, 'general_info']}\nSố phòng ngủ ước lượng là:"

            response = interface.query_model(prompt, max_new_tokens=max_new_tokens, temperature=temperature, top_p=top_p)
            room_number = check_valid_room_number_response(response)
            # print(f"Index: {idx}, Parsed: {room_number}")
            
            total_processed += 1
            
            if room_number is not None:
                df_new.loc[idx, 'category'] = room_number
            else:
                df_new.loc[idx, 'retry_count'] = df_new.loc[idx, 'retry_count'] + 1
                total_errors += 1
            
            error_rate = (total_errors / total_processed) * 100
            
            if idx % 10 == 0:
                progress_bar.set_description(f"Retry {i+1}/{max_retries} | Error rate: {error_rate:.2f}%")

In [ ]:
df_new['category'].plot(kind = 'hist')

In [ ]:
df.to_csv('../data/guland_hanoi_listings_with_bedrooms_arcgis4.csv', index=False)